In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [2]:
x = sp.symbols('x')
cap = 2000 # total emission cap
Regulator6 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator6)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator6)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator6)
sector4 = Sector('plastic', price_demand_function= 400 - 0.5*x, free_emission_multiplier= 0, regulator= Regulator6)
sector5 = Sector('glass', price_demand_function= 300 - 0.4*x, free_emission_multiplier= 0, regulator= Regulator6)
country1 = Country('DE', 1, regulator= Regulator6)
country2 = Country('FI', 0.5, regulator= Regulator6)
country3 = Country('GR', size= 0.1, regulator= Regulator6)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 10*x+ 2*x**2 + 0.1*x**3 , 0, 0, 0, regulator= Regulator6)
firm2 = Firm('firm2', 1, 2, x*0, 11*x+ 3*x**2 + 0.2*x**3, 0, 0, 0, regulator= Regulator6)
firm3 = Firm('firm3', 1, 3, x*0, 5*x+ 4*x**2 + 5*x**3 , 0, 0, 0, regulator= Regulator6)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator6)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator6)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator6)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator6)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator6)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator6)

Regulator6.optimization_with_least_squares()

pp = sp.symbols('pp') # permit price
q1, q2 = sp.symbols('q1 q2') # quantity of firm1 and firm2
x1, x2 = sp.symbols('x1 x2') # emission of firm1 and firm2



profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - 100*(q2 - x2)**2 - pp*x2

# First order conditions
foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Solve for q1, q2, x1, x2
solutions = sp.solve([foc1, foc2, foc3, foc4, x1 + x2 - cap], (q1, q2, x1, x2, pp))

print(solutions)

#Now again using a least squares approach and scipy optimization
print(foc1,"|", foc2,"|", foc3,"|", foc4)






Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-02
Firm firm1 has profit: -pp*x1 + q1*(-0.1*q1 - 0.1*q2 - 0.1*q3 + 100) - 10*q1 + 10*x1 - 0.1*(q1 - x1)**3 - 2*(q1 - x1)**2
FOD: -4.2*q1 - 0.1*q2 - 0.1*q3 + 4*x1 - 0.3*(q1 - x1)**2 + 90
FED: -pp + 4*q1 - 4*x1 + 0.3*(q1 - x1)**2 + 10
Firm firm2 has profit: -pp*x2 + q2*(-0.1*q1 - 0.1*q2 - 0.1*q3 + 100) - 11*q2 + 11*x2 - 0.2*(q2 - x2)**3 - 3*(q2 - x2)**2
FOD: -0.1*q1 - 6.2*q2 - 0.1*q3 + 6*x2 - 0.6*(q2 - x2)**2 + 89
FED: -pp + 6*q2 - 6*x2 + 0.6*(q2 - x2)**2 + 11
Firm firm3 has profit: -pp*x3 + q3*(-0.1*q1 - 0.1*q2 - 0.1*q3 + 100) - 5*q3 + 5*x3 - 5*(q3 - x3)**3 - 4*(q3 - x3)**2
FOD: -0.1*q1 - 0.1*q2 - 8.2*q3 + 8*x3 - 15*(q3 - x3)**2 + 95
FED: -pp + 8*q3 - 8*x3 + 15*(q3 - x3)**2 + 5
Firm firm4 has profit: -pp*x4 + q4*(-0.1*q4 - 0.1*q5 - 0.1*q6 + 150) - 7*q4 + 7*x4 - 3*(q4 - x4)**3 - 5*(q4 - x4)**2
FOD: -10.2*q4 - 0.1*q5 - 0.1*q6 + 10*x4 - 9*(q4 - x4)**2 + 143
FED: -pp + 10*q4 - 10*x4 + 9*(q4 - x4)**2 + 7


Παρακάτω λύνουμε το ίδιο βάζοντας ουσιαστικά κάθε μεταβλητή που μας ενδιαφέρει στο τετράγωνο. και ελαχιστοποιώντας το άθροισμά του. 

In [3]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1', lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2', lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp', lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn = foc1**2 + foc2**2 + foc3**2 + foc4**2 + (cap - x1 - x2)**2
example.setObjective(obj_fn, GRB.MINIMIZE)

example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x083a456c
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 4e+03]
  QObjective range [2e+00, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 2e+03]
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 6 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 2.000e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.51230949

Εδώ θα κάνουμε το ίδιο, αλλά αυτήν την φορά θα κάνουμε την δική μου ιδέα με τα constrains και το αθροισμα αλλού.

In [4]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1', lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2', lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp', lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn =   x1 + x2
example.setObjective(obj_fn, GRB.MAXIMIZE)
example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)
example.addConstr(foc1 == 0)
example.addConstr(foc2 == 0)
example.addConstr(foc3 == 0)
example.addConstr(foc4 == 0)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 18 nonzeros
Model fingerprint: 0xbbb49918
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 3 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0005000e+02   7.262626e+00   0.000000e+00      0s
       2    6.6666667e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  6.666666667e+01
q1: 33.3333
q2: 33.3333
x1: 33.3333
x2: 33.3333
pp: 0


In [5]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1', lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2', lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp', lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn =   (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
example.setObjective(obj_fn, GRB.MAXIMIZE)
example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)
example.addConstr(foc1 == 0)
example.addConstr(foc2 == 0)
example.addConstr(foc3 == 0)
example.addConstr(foc4 == 0)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 18 nonzeros
Model fingerprint: 0xc4de330b
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [1e+01, 1e+01]
  QObjective range [2e-01, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Continuous model is non-convex -- solving as a MIP

Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 11 rows, 11 columns, 33 nonzeros
Presolved model has 2 quadratic constraint(s)
Presolved model has 3 bilinear constraint(s)
Variable types: 11 continuous, 0 integer (0 binary)
Found heuristic solution: objective 111.1111111

Root relaxation: objective 8.149979e+03, 4 iterations, 0.00 seconds (0.00 work units)

 

In [6]:
# How would my optimzer handle this?
x = sp.symbols('x')
Regulator6 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector("sector1", 10-0.1*x, 0, Regulator6)
fra = Country("France", 50, Regulator6)
# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 2*x**2 , 0, 0, 0, regulator= Regulator6)
firm2 = Firm('firm2', 1, 1, x*0, 100*x**2 , 0, 0, 0, regulator= Regulator6)

Regulator6.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision=0.0001)
for firm in Regulator6.firm_registry.values():
    print(firm.name, firm.actual_output, firm.emission)
# Από ότι φαίνεται, το ίδιο. 

Permit price: 0.488281 price = 0.5, cap = 2000, second_stage = False, a = 1.0000Permit price: 0.48828125 and total emission: 0 and emission cap 2000
firm1 0 0
firm2 0 0
